In [1]:
import numpy as np
import pandas as pd

# Bureau 

In [2]:
bureau = pd.read_csv('bureau.csv')
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,138615,6745057,Closed,currency 1,-1523,0,-1342.0,-1369.0,NaN,0,26301.96,0.0,NaN,0.0,Consumer credit,-800,NaN
1048571,369065,6745058,Closed,currency 1,-1650,0,-918.0,-918.0,NaN,0,127399.50,0.0,0.0,0.0,Consumer credit,-864,NaN
1048572,369065,6745059,Active,currency 1,-195,0,1419.0,NaN,NaN,0,639000.00,594391.5,0.0,0.0,Consumer credit,-6,0.0
1048573,369065,6745060,Closed,currency 1,-379,0,-6.0,-282.0,0.0,0,67500.00,0.0,0.0,0.0,Consumer credit,-282,19039.5


In [3]:
bureau[bureau['SK_ID_BUREAU'] == 6842884]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY


In [4]:
bureau = bureau.drop_duplicates()
bureau.shape

(1048575, 17)

In [5]:
bureau.select_dtypes(include = object).isnull().sum()

CREDIT_ACTIVE      0
CREDIT_CURRENCY    0
CREDIT_TYPE        0
dtype: int64

In [6]:
bureau.select_dtypes(exclude = object).isnull().sum()

SK_ID_CURR                     0
SK_ID_BUREAU                   0
DAYS_CREDIT                    0
CREDIT_DAY_OVERDUE             0
DAYS_CREDIT_ENDDATE        65119
DAYS_ENDDATE_FACT         389308
AMT_CREDIT_MAX_OVERDUE    690835
CNT_CREDIT_PROLONG             0
AMT_CREDIT_SUM                 7
AMT_CREDIT_SUM_DEBT       157335
AMT_CREDIT_SUM_LIMIT      365062
AMT_CREDIT_SUM_OVERDUE         0
DAYS_CREDIT_UPDATE             0
AMT_ANNUITY               729443
dtype: int64

In [7]:
bureau.fillna(0, inplace =True)
bureau.isnull().sum()

SK_ID_CURR                0
SK_ID_BUREAU              0
CREDIT_ACTIVE             0
CREDIT_CURRENCY           0
DAYS_CREDIT               0
CREDIT_DAY_OVERDUE        0
DAYS_CREDIT_ENDDATE       0
DAYS_ENDDATE_FACT         0
AMT_CREDIT_MAX_OVERDUE    0
CNT_CREDIT_PROLONG        0
AMT_CREDIT_SUM            0
AMT_CREDIT_SUM_DEBT       0
AMT_CREDIT_SUM_LIMIT      0
AMT_CREDIT_SUM_OVERDUE    0
CREDIT_TYPE               0
DAYS_CREDIT_UPDATE        0
AMT_ANNUITY               0
dtype: int64

In [8]:
bureau['Duration_planned'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT'] 
bureau['Duration_real']    = bureau['DAYS_ENDDATE_FACT'] - bureau['DAYS_CREDIT']

In [9]:
bureau = bureau.drop(bureau[bureau['Duration_planned'] == 0].index) #500 rows had duration of 0

In [10]:
# Nagative days_early_paid means: prolonged bureau
bureau['Days_early_paidoff'] =  bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

In [11]:
# Weighted sum of credit a person borrowed (per days) (later aggregation: Weighted sume of credit debt per day per each credit)
bureau['AMT_weightdebt_duration'] = bureau['AMT_CREDIT_SUM_DEBT']/bureau['Duration_planned']

In [12]:
# 'AMT_CREDIT_SUM_OVERDUE': Current amount overdue on credit 
bureau['AMT_Overdue_duration'] = bureau['AMT_CREDIT_SUM_OVERDUE']/bureau['Duration_planned'] 

In [13]:
# Maximal amount overdue so far 
bureau['AMT_Maxoverdue_duration'] = bureau['AMT_CREDIT_MAX_OVERDUE']/bureau['Duration_planned'] 

In [14]:
# Default credit day overdue more than 270 days
bureau['Defaulted'] = np.where(bureau['CREDIT_DAY_OVERDUE'] > 270, 1, 0)
bureau['AMT_defaulted'] = bureau['Defaulted']*bureau['AMT_CREDIT_SUM_DEBT']
# np.count_nonzero(bureau['Default'])

In [15]:
# number of current credit active: sum of credit active atttibute
mapping = {'Closed':0, 'Active': 1}
bureau['CREDIT_ACTIVE'] = bureau['CREDIT_ACTIVE'].map(mapping)
bureau['CREDIT_ACTIVE']

0          0.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
1048570    0.0
1048571    0.0
1048572    1.0
1048573    0.0
1048574    0.0
Name: CREDIT_ACTIVE, Length: 1048012, dtype: float64

In [16]:
stats = bureau.sort_values('DAYS_CREDIT', ascending = False).groupby('SK_ID_CURR') \
        .agg({'AMT_CREDIT_SUM_DEBT': ['count' ,'sum', 'mean'], 
              'AMT_weightdebt_duration': ['sum', 'mean'], 
              'AMT_Overdue_duration': [ 'sum', 'mean'], 
              'AMT_Maxoverdue_duration': ['mean'], 
              'Days_early_paidoff': [ 'sum', 'mean'],
              'Defaulted':[ 'sum', 'mean'], 
              'AMT_defaulted':[ 'sum', 'mean'],
              'CREDIT_ACTIVE': 'sum'}) 
stats

AMT_CREDIT_SUM_DEBT                             \
                         count         sum           mean   
SK_ID_CURR                                                  
100001                       7   596686.50   85240.928571   
100002                       6        0.00       0.000000   
100003                       4        0.00       0.000000   
100004                       2        0.00       0.000000   
100008                       2   240057.00  120028.500000   
...                        ...         ...            ...   
456243                       5  1247454.00  249490.800000   
456247                       7  2193390.00  313341.428571   
456249                       4        0.00       0.000000   
456253                       4  1795833.00  448958.250000   
456255                      11  1534913.01  139537.546364   

           AMT_weightdebt_duration              AMT_Overdue_duration       \
                               sum         mean                  sum mean   
SK_ID_CURR                                                                  
100001                  432.612431    61.801776                  0.0  0.0   
100002                    0.000000     0.000000                  0.0  0.0   
100003                    0.000000     0.000000                  0.0  0.0   
100004                    0.000000     0.000000                  0.0  0.0   
100008                  437.262295   218.631148                  0.0  0.0   
...                            ...          ...                  ...  ...   
456243                 8100.350649  1620.070130                  0.0  0.0   
456247                  200.035568    28.576510                  0.0  0.0   
456249                    0.000000     0.000000                  0.0  0.0   
456253                 1020.773873   255.193468                  0.0  0.0   
456255                 1084.646559    98.604233                  0.0  0.0   

           AMT_Maxoverdue_duration Days_early_paidoff              Defaulted  \
                              mean                sum         mean       sum   
SK_ID_CURR                                                                     
100001                    0.000000             3879.0   554.142857         0   
100002                    3.852765              282.0    47.000000         0   
100003                    0.000000             1114.0   278.500000         0   
100004                    0.000000               88.0    44.000000         0   
100008                    0.000000              646.0   323.000000         0   
...                            ...                ...          ...       ...   
456243                    0.000000             -299.0   -59.800000         0   
456247                   25.283572            13921.0  1988.714286         0   
456249                   17.348901              112.0    28.000000         0   
456253                    0.000000             2710.0   677.500000         0   
456255                    4.598421            41354.0  3759.454545         0   

                AMT_defaulted      CREDIT_ACTIVE  
           mean           sum mean           sum  
SK_ID_CURR                                        
100001      0.0           0.0  0.0           3.0  
100002      0.0           0.0  0.0           1.0  
100003      0.0           0.0  0.0           1.0  
100004      0.0           0.0  0.0           0.0  
100008      0.0           0.0  0.0           1.0  
...         ...           ...  ...           ...  
456243      0.0           0.0  0.0           2.0  
456247      0.0           0.0  0.0           2.0  
456249      0.0           0.0  0.0           0.0  
456253      0.0           0.0  0.0           2.0  
456255      0.0           0.0  0.0           5.0  

[218273 rows x 15 columns]

In [17]:
stats.columns = stats.columns.map('_'.join)
stats
# train = pd.merge(train, stats, on='SK_ID_CURR', how = 'left')

,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,Defaulted_sum,Defaulted_mean,AMT_defaulted_sum,AMT_defaulted_mean,CREDIT_ACTIVE_sum
SK_ID_CURR,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,0,0.0,0.0,0.0,3.0
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,0,0.0,0.0,0.0,1.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,0,0.0,0.0,0.0,1.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,0,0.0,0.0,0.0,0.0
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,0,0.0,0.0,0.0,2.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,0,0.0,0.0,0.0,2.0
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,0,0.0,0.0,0.0,0.0


In [18]:
# atest = atest.merge(stats, on='SK_ID_CURR', how='left')
# atest

In [19]:
# nth() method is not working with groupby + aggregation 
columns = ['SK_ID_CURR', 'DAYS_CREDIT', 'AMT_CREDIT_SUM_DEBT', 'AMT_weightdebt_duration', 'AMT_Overdue_duration', \
          'Days_early_paidoff','Defaulted', 'AMT_defaulted'] 
stats0 = bureau.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby('SK_ID_CURR').nth(0) # same as first
stats1 = bureau.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby('SK_ID_CURR').nth(1)
stats2 = bureau.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby('SK_ID_CURR').nth(2)
stats3 = bureau.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby('SK_ID_CURR').nth(3)

In [20]:
from functools import reduce
data_frames = [stats0, stats1, stats2, stats3]
lags = reduce(lambda  left,right: pd.concat( [left,right], axis=1, sort= False), data_frames)
col=[]
for i in range(4):
    col += ['DAYS_CREDIT'+str(i), 'AMT_CREDIT_SUM_DEBT'+ str(i), 'AMT_weightdebt_duration'+ str(i), \
                    'AMT_Overdue_duration'+ str(i), 'Days_early_paidoff'+ str(i),'Defaulted'+ str(i), 'AMT_defaulted'+ str(i)] 
lags.columns = col
lags

,DAYS_CREDIT0,AMT_CREDIT_SUM_DEBT0,AMT_weightdebt_duration0,AMT_Overdue_duration0,Days_early_paidoff0,Defaulted0,AMT_defaulted0,DAYS_CREDIT1,AMT_CREDIT_SUM_DEBT1,AMT_weightdebt_duration1,...,Days_early_paidoff2,Defaulted2,AMT_defaulted2,DAYS_CREDIT3,AMT_CREDIT_SUM_DEBT3,AMT_weightdebt_duration3,AMT_Overdue_duration3,Days_early_paidoff3,Defaulted3,AMT_defaulted3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-49,373239.00,204.290640,0.0,1778.0,0,0.0,-320.0,110281.5,150.863885,...,902.0,0.0,0.0,-857.0,0.0,0.000000,0.0,61.0,0.0,0.0
100002,-103,0.00,0.000000,0.0,0.0,0,0.0,-476.0,0.0,0.000000,...,121.0,0.0,0.0,-1121.0,0.0,0.000000,0.0,0.0,0.0,0.0
100003,-606,0.00,0.000000,0.0,1216.0,0,0.0,-775.0,0.0,0.000000,...,0.0,0.0,0.0,-2586.0,0.0,0.000000,0.0,-303.0,0.0,0.0
100004,-408,0.00,0.000000,0.0,0.0,0,0.0,-1326.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100008,-78,240057.00,437.262295,0.0,471.0,0,0.0,-1097.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,-154,1247454.00,8100.350649,0.0,0.0,0,0.0,-263.0,0.0,0.000000,...,887.0,0.0,0.0,-796.0,0.0,0.000000,0.0,0.0,0.0,0.0
456247,-287,0.00,0.000000,0.0,0.0,0,0.0,-309.0,0.0,0.000000,...,1760.0,0.0,0.0,-712.0,2193390.0,200.035568,0.0,10253.0,0.0,0.0
456249,-1250,0.00,0.000000,0.0,0.0,0,0.0,-1454.0,0.0,0.000000,...,40.0,0.0,0.0,-1911.0,0.0,0.000000,0.0,13.0,0.0,0.0


In [21]:
bureau_info = pd.merge(stats, lags, on= 'SK_ID_CURR', how ='outer')
bureau_info.fillna(0, inplace= True)
bureau_info

,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,...,Days_early_paidoff2,Defaulted2,AMT_defaulted2,DAYS_CREDIT3,AMT_CREDIT_SUM_DEBT3,AMT_weightdebt_duration3,AMT_Overdue_duration3,Days_early_paidoff3,Defaulted3,AMT_defaulted3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,...,902.0,0.0,0.0,-857.0,0.0,0.000000,0.0,61.0,0.0,0.0
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,...,121.0,0.0,0.0,-1121.0,0.0,0.000000,0.0,0.0,0.0,0.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,...,0.0,0.0,0.0,-2586.0,0.0,0.000000,0.0,-303.0,0.0,0.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,...,887.0,0.0,0.0,-796.0,0.0,0.000000,0.0,0.0,0.0,0.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,...,1760.0,0.0,0.0,-712.0,2193390.0,200.035568,0.0,10253.0,0.0,0.0
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,...,40.0,0.0,0.0,-1911.0,0.0,0.000000,0.0,13.0,0.0,0.0


In [22]:
# train = train.merge(lags, on= 'SK_ID_CURR', how= 'left')
# atest = atest.merge(lags, on= 'SK_ID_CURR', how= 'left')

In [23]:
# multiindex selection: stats['DAYS_CREDIT','first']
# stats.columns

In [24]:
bureau['CREDIT_TYPE'].unique()

array(['Consumer credit', 'Credit card', 'Mortgage', 'Car loan',
       'Microloan', 'Loan for working capital replenishment',
       'Loan for business development', 'Real estate loan',
       'Unknown type of loan', 'Another type of loan',
       'Cash loan (non-earmarked)', 'Loan for the purchase of equipment',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [25]:
# Filter 6 categories of loans
Filter = (bureau['CREDIT_TYPE'] == 'Mortgage')  | (bureau['CREDIT_TYPE'] == 'Real estate loan') \
         | (bureau['CREDIT_TYPE'] == 'Car loan') \
         | (bureau['CREDIT_TYPE'] == 'Loan for business development') \
         | (bureau['CREDIT_TYPE'] == 'Loan for the purchase of equipment') \
         | (bureau['CREDIT_TYPE'] == 'Cash loan (non-earmarked)')\
         | (bureau['CREDIT_TYPE'] == 'Loan for working capital replenishment') 
btype = bureau[Filter].copy()
btype

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,DAYS_CREDIT_UPDATE,AMT_ANNUITY,Duration_planned,Duration_real,Days_early_paidoff,AMT_weightdebt_duration,AMT_Overdue_duration,AMT_Maxoverdue_duration,Defaulted,AMT_defaulted
12,162297,5714474,1.0,currency 1,-277,0,5261.0,0.0,0.000,0,...,-31,0.0,5538.0,277.0,5261.0,0.000000,0.0,0.000000,0,0.0
26,222183,5714497,1.0,currency 1,-315,0,1512.0,0.0,88821.000,0,...,-32,0.0,1827.0,315.0,1512.0,0.000000,0.0,48.615764,0,0.0
81,234931,5714561,0.0,currency 1,-2638,0,-1208.0,-1518.0,18293.265,0,...,-704,0.0,1430.0,1120.0,310.0,0.000000,0.0,12.792493,0,0.0
108,419892,5714596,1.0,currency 1,-381,0,0.0,0.0,0.000,0,...,-27,0.0,381.0,381.0,0.0,7190.610236,0.0,0.000000,0,0.0
288,181326,5714830,0.0,currency 1,-1369,0,457.0,-221.0,0.000,0,...,-221,0.0,1826.0,1148.0,678.0,0.000000,0.0,0.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048445,332068,6744917,1.0,currency 1,-1198,0,629.0,0.0,0.000,0,...,-47,0.0,1827.0,1198.0,629.0,348.162562,0.0,0.000000,0,0.0
1048465,129847,6744939,0.0,currency 1,-482,0,1344.0,-157.0,59247.000,0,...,-157,0.0,1826.0,325.0,1501.0,0.000000,0.0,32.446331,0,0.0
1048473,373050,6744950,1.0,currency 1,-1113,0,325.0,0.0,0.000,0,...,-6,23557.5,1438.0,1113.0,325.0,169.328929,0.0,0.000000,0,0.0
1048505,303014,6744987,0.0,currency 1,-1588,0,238.0,-589.0,0.000,0,...,-589,90247.5,1826.0,999.0,827.0,0.000000,0.0,0.000000,0,0.0


In [26]:
btype['Defaulted'] = np.where(btype['CREDIT_DAY_OVERDUE'] > 270, 1, 0)
btype['AMT_defaulted'] = btype['Defaulted']*btype['AMT_CREDIT_SUM_DEBT']

In [27]:
# first(DAYS_CREDIT) is the last credit open for each bureau credit, last(Days_Credit) is the first day 
# first/last credit bureau open, close,times credit deafult, amount credit defaulted 
bt_stat  = btype.sort_values('DAYS_CREDIT', ascending = False).groupby(['SK_ID_CURR','CREDIT_TYPE'])\
     .agg({ 'DAYS_CREDIT':'last', 'AMT_CREDIT_SUM_DEBT': ['count', 'sum', 'mean'],\
           'CREDIT_ACTIVE': 'sum' ,'Defaulted':'sum', 'AMT_defaulted':['sum']})
bt_stat.columns = bt_stat.columns.map('_'.join)
bt_stat

,,DAYS_CREDIT_last,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,CREDIT_ACTIVE_sum,Defaulted_sum,AMT_defaulted_sum
SK_ID_CURR,CREDIT_TYPE,,,,,,,
100013,Car loan,-2005,1,0.000,0.000,0.0,0,0.0
100029,Mortgage,-2806,2,3682287.000,1841143.500,1.0,0,0.0
100039,Car loan,-1206,2,0.000,0.000,0.0,0,0.0
100055,Car loan,-1204,1,1210297.500,1210297.500,1.0,0,0.0
100057,Car loan,-787,1,0.000,0.000,0.0,0,0.0
...,...,...,...,...,...,...,...,...
456186,Mortgage,-375,1,3841653.015,3841653.015,1.0,0,0.0
456219,Car loan,-821,1,0.000,0.000,1.0,0,0.0
456223,Mortgage,-218,1,0.000,0.000,0.0,0,0.0


In [28]:
mortgage = bt_stat[bt_stat.index.isin(['Mortgage'], level='CREDIT_TYPE')]
mortgage.index = mortgage.index.droplevel('CREDIT_TYPE')
realestate = bt_stat[bt_stat.index.isin(['Real estate loan'], level='CREDIT_TYPE')]
realestate.index = realestate.index.droplevel('CREDIT_TYPE')
carloan = bt_stat[bt_stat.index.isin(['Car loan'], level='CREDIT_TYPE')]
carloan.index = carloan.index.droplevel('CREDIT_TYPE')
loanbusiness = bt_stat[bt_stat.index.isin(['Loan for business development'], level='CREDIT_TYPE')]
loanbusiness.index = loanbusiness.index.droplevel('CREDIT_TYPE')
loanpurchase = bt_stat[bt_stat.index.isin(['Loan for the purchase of equipment'], level='CREDIT_TYPE')]
loanpurchase.index = loanpurchase.index.droplevel('CREDIT_TYPE')
cashloan = bt_stat[bt_stat.index.isin(['Cash loan (non-earmarked)'], level='CREDIT_TYPE')]
cashloan.index = cashloan.index.droplevel('CREDIT_TYPE')
workingloan = bt_stat[bt_stat.index.isin(['Loan for working capital replenishment'], level='CREDIT_TYPE')]
workingloan.index = workingloan.index.droplevel('CREDIT_TYPE')

In [29]:
dataframes = [mortgage, realestate, carloan, loanbusiness, loanpurchase, cashloan, workingloan]
credit_type = reduce(lambda left, right: pd.merge(left,right, on = 'SK_ID_CURR', how= 'outer'), dataframes)
types = ['mortgage', 'realestate', 'carloan', 'loanbusiness', 'loanpurchase', 'cashloan', 'workingloan']
columns =[]
for s in types:
        columns += ['DAYS_CREDIT_last_' + s , 'AMT_CREDIT_SUM_DEBT_count_'+ s,   \
        'AMT_CREDIT_SUM_DEBT_sum_' + s , 'AMT_CREDIT_SUM_DEBT_mean_' + s ,   \
        'CREDIT_ACTIVE_sum_x_' + s , 'Defaulted_sum_x_' + s , 'AMT_defaulted_sum_' + s]
credit_type.columns = columns
credit_type = credit_type.fillna(0)
credit_type

,DAYS_CREDIT_last_mortgage,AMT_CREDIT_SUM_DEBT_count_mortgage,AMT_CREDIT_SUM_DEBT_sum_mortgage,AMT_CREDIT_SUM_DEBT_mean_mortgage,CREDIT_ACTIVE_sum_x_mortgage,Defaulted_sum_x_mortgage,AMT_defaulted_sum_mortgage,DAYS_CREDIT_last_realestate,AMT_CREDIT_SUM_DEBT_count_realestate,AMT_CREDIT_SUM_DEBT_sum_realestate,...,CREDIT_ACTIVE_sum_x_cashloan,Defaulted_sum_x_cashloan,AMT_defaulted_sum_cashloan,DAYS_CREDIT_last_workingloan,AMT_CREDIT_SUM_DEBT_count_workingloan,AMT_CREDIT_SUM_DEBT_sum_workingloan,AMT_CREDIT_SUM_DEBT_mean_workingloan,CREDIT_ACTIVE_sum_x_workingloan,Defaulted_sum_x_workingloan,AMT_defaulted_sum_workingloan
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100029,-2806.0,2.0,3682287.0,1841143.5,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100057,-1511.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100082,-1479.0,1.0,2188683.0,2188683.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100129,-914.0,1.0,10048887.0,10048887.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100145,-777.0,1.0,2915271.0,2915271.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1526.0,2.0,0.0,0.0,0.0,0.0,0.0
447107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-217.0,1.0,180450.0,180450.0,1.0,0.0,0.0
448503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1827.0,2.0,0.0,0.0,0.0,0.0,0.0


In [30]:
bureau_info.merge(credit_type, on ='SK_ID_CURR', how= 'outer')
bureau_info

,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,...,Days_early_paidoff2,Defaulted2,AMT_defaulted2,DAYS_CREDIT3,AMT_CREDIT_SUM_DEBT3,AMT_weightdebt_duration3,AMT_Overdue_duration3,Days_early_paidoff3,Defaulted3,AMT_defaulted3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,...,902.0,0.0,0.0,-857.0,0.0,0.000000,0.0,61.0,0.0,0.0
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,...,121.0,0.0,0.0,-1121.0,0.0,0.000000,0.0,0.0,0.0,0.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,...,0.0,0.0,0.0,-2586.0,0.0,0.000000,0.0,-303.0,0.0,0.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,...,887.0,0.0,0.0,-796.0,0.0,0.000000,0.0,0.0,0.0,0.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,...,1760.0,0.0,0.0,-712.0,2193390.0,200.035568,0.0,10253.0,0.0,0.0
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,...,40.0,0.0,0.0,-1911.0,0.0,0.000000,0.0,13.0,0.0,0.0


In [31]:
# train = train.merge(bt_stat, on= 'SK_ID_CURR', how= 'left')
# atest = atest.merge(bt_stat, on= 'SK_ID_CURR', how= 'left')

In [32]:
# del bt_stat

In [33]:
# Flatten last four lags for each credit type
columns   = ['SK_ID_CURR','DAYS_CREDIT', 'AMT_CREDIT_SUM_DEBT', 'CREDIT_ACTIVE','Defaulted', 'AMT_defaulted']
bt_stat0  = btype.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby(['SK_ID_CURR']).nth(0)
bt_stat1  = btype.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby(['SK_ID_CURR']).nth(1)
bt_stat2  = btype.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby(['SK_ID_CURR']).nth(2)
bt_stat3  = btype.sort_values('DAYS_CREDIT', ascending = False)[columns].groupby(['SK_ID_CURR']).nth(3)

In [34]:
from functools import reduce
data_frames = [bt_stat0, bt_stat1, bt_stat2, bt_stat3]
blags = reduce(lambda  left,right: pd.concat( [left,right], axis=1, sort= False), data_frames)
col=[]
for i in range(4):
    col += ['DAYS_CREDIT'+str(i), 'AMT_CREDIT_SUM_DEBT'+str(i), 'CREDIT_ACTIVE'+str(i),'Default'+str(i), 'AMT_defaulted'+str(i)] 
blags.columns = col
blags.fillna(0, inplace= True)
blags

,DAYS_CREDIT0,AMT_CREDIT_SUM_DEBT0,CREDIT_ACTIVE0,Default0,AMT_defaulted0,DAYS_CREDIT1,AMT_CREDIT_SUM_DEBT1,CREDIT_ACTIVE1,Default1,AMT_defaulted1,DAYS_CREDIT2,AMT_CREDIT_SUM_DEBT2,CREDIT_ACTIVE2,Default2,AMT_defaulted2,DAYS_CREDIT3,AMT_CREDIT_SUM_DEBT3,CREDIT_ACTIVE3,Default3,AMT_defaulted3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100013,-2005,0.000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100029,-110,3682287.000,1.0,0,0.0,-2806.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100039,-557,0.000,0.0,0,0.0,-1206.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100055,-1204,1210297.500,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100057,-787,0.000,0.0,0,0.0,-1511.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456186,-375,3841653.015,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456219,-821,0.000,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456223,-218,0.000,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
bureau_info.merge(blags, on = 'SK_ID_CURR', how= 'outer')
bureau_info

,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,...,Days_early_paidoff2,Defaulted2,AMT_defaulted2,DAYS_CREDIT3,AMT_CREDIT_SUM_DEBT3,AMT_weightdebt_duration3,AMT_Overdue_duration3,Days_early_paidoff3,Defaulted3,AMT_defaulted3
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,...,902.0,0.0,0.0,-857.0,0.0,0.000000,0.0,61.0,0.0,0.0
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,...,121.0,0.0,0.0,-1121.0,0.0,0.000000,0.0,0.0,0.0,0.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,...,0.0,0.0,0.0,-2586.0,0.0,0.000000,0.0,-303.0,0.0,0.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,...,887.0,0.0,0.0,-796.0,0.0,0.000000,0.0,0.0,0.0,0.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,...,1760.0,0.0,0.0,-712.0,2193390.0,200.035568,0.0,10253.0,0.0,0.0
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,...,40.0,0.0,0.0,-1911.0,0.0,0.000000,0.0,13.0,0.0,0.0


In [36]:
# train = train.merge(blags, on= 'SK_ID_CURR', how= 'left')
# del btype, bt_stat0,bt_stat1,bt_stat2,bt_stat3,blags

In [37]:
# Total outstanding debt on credit cards from bureau for comparison with Credit Card at home credit table 
CCdebt_bureau = bureau[bureau['CREDIT_TYPE'] == 'Credit card'].sort_values('DAYS_CREDIT', ascending = False)\
                .groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM_DEBT':['count','sum', 'mean', 'first']})
CCdebt_bureau.columns = CCdebt_bureau.columns.map('_'.join)
CCdebt_bureau

,AMT_CREDIT_SUM_DEBT_count,AMT_CREDIT_SUM_DEBT_sum,AMT_CREDIT_SUM_DEBT_mean,AMT_CREDIT_SUM_DEBT_first
SK_ID_CURR,,,,
100002,3,0.000,0.000,0.000
100003,2,0.000,0.000,0.000
100011,1,0.000,0.000,0.000
100017,1,0.000,0.000,0.000
100019,1,0.000,0.000,0.000
...,...,...,...,...
456234,1,-385.785,-385.785,-385.785
456239,3,737680.500,245893.500,0.000
456243,1,0.000,0.000,0.000


In [38]:
bureau_info = bureau_info.merge(CCdebt_bureau, on= 'SK_ID_CURR', how = 'outer')
bureau_info

,AMT_CREDIT_SUM_DEBT_count_x,AMT_CREDIT_SUM_DEBT_sum_x,AMT_CREDIT_SUM_DEBT_mean_x,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,...,AMT_CREDIT_SUM_DEBT3,AMT_weightdebt_duration3,AMT_Overdue_duration3,Days_early_paidoff3,Defaulted3,AMT_defaulted3,AMT_CREDIT_SUM_DEBT_count_y,AMT_CREDIT_SUM_DEBT_sum_y,AMT_CREDIT_SUM_DEBT_mean_y,AMT_CREDIT_SUM_DEBT_first
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,...,0.0,0.000000,0.0,61.0,0.0,0.0,NaN,NaN,NaN,NaN
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.00,0.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,...,0.0,0.000000,0.0,-303.0,0.0,0.0,2.0,0.0,0.00,0.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,...,2193390.0,200.035568,0.0,10253.0,0.0,0.0,NaN,NaN,NaN,NaN
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,...,0.0,0.000000,0.0,13.0,0.0,0.0,NaN,NaN,NaN,NaN


In [39]:
# train = train.merge(CCdebt_bureau, on= 'SK_ID_CURR', how= 'left')
# del CCdebt_bureau, bureau

In [40]:
bureaubal = pd.read_csv('bureau_balance.csv')
bureaubal

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [41]:
bureaubal[bureaubal['SK_ID_BUREAU'] == 5714462] # first zero and first 1

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS


In [42]:
bureaubal.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [43]:
# Last close is obtained by first 0 , last delinquency is obtained from first 1 in the MONTS_BALANCE
first = bureaubal.groupby(['SK_ID_BUREAU','STATUS'], as_index=False).first()
left  = first[(first['STATUS'] == '0')][['SK_ID_BUREAU','MONTHS_BALANCE']]
right = first[(first['STATUS'] == '1')][['SK_ID_BUREAU','MONTHS_BALANCE']]
first = pd.merge(left,right, on= 'SK_ID_BUREAU', how='left').fillna(0)
first.columns = ['SK_ID_BUREAU','Months_latest_open','latest_delinq_balance']
first

,SK_ID_BUREAU,Months_latest_open,latest_delinq_balance
0,5001710,-48,0.0
1,5001711,-1,0.0
2,5001712,-9,0.0
3,5001716,-39,0.0
4,5001717,-5,0.0
...,...,...,...
685572,6842884,-38,0.0
685573,6842885,-12,0.0
685574,6842886,-25,0.0
685575,6842887,-31,0.0


In [44]:
# d= bureaubal[(bureaubal['STATUS'] != 'X') & (bureaubal['STATUS'] != 'C')].groupby(['SK_ID_BUREAU','STATUS']).agg({'STATUS':'count'})
# d = d.groupby(level='SK_ID_BUREAU').apply(lambda x: x/x.sum())
# d.tail(10)

In [45]:
delinquency = pd.get_dummies(bureaubal[(bureaubal['STATUS'] != 'X') & (bureaubal['STATUS'] != 'C')])
# delinquency = delinquency.groupby('SK_ID_BUREAU').agg(['sum', 'count', 'mean', 'last']) # STATUS_1 means 1-30 days delinquency
delinquency = delinquency.groupby('SK_ID_BUREAU').agg({'STATUS_0':'mean','STATUS_1':'mean','STATUS_2':'mean','STATUS_3':'mean',
                                                     'STATUS_4':'mean','STATUS_5':'mean'}) # STATUS_1 means 1-30 days delinquency
delinquency

,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5
SK_ID_BUREAU,,,,,,
5001710,1.0,0.0,0.0,0.0,0.0,0.0
5001711,1.0,0.0,0.0,0.0,0.0,0.0
5001712,1.0,0.0,0.0,0.0,0.0,0.0
5001716,1.0,0.0,0.0,0.0,0.0,0.0
5001717,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
6842884,1.0,0.0,0.0,0.0,0.0,0.0
6842885,0.5,0.0,0.0,0.0,0.0,0.5
6842886,1.0,0.0,0.0,0.0,0.0,0.0


In [46]:
ccbalance = pd.merge(first, delinquency, on='SK_ID_BUREAU', how = 'inner').fillna(0)
ccbalance

,SK_ID_BUREAU,Months_latest_open,latest_delinq_balance,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5
0,5001710,-48,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5001711,-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,5001712,-9,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,5001716,-39,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5001717,-5,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
685572,6842884,-38,0.0,1.0,0.0,0.0,0.0,0.0,0.0
685573,6842885,-12,0.0,0.5,0.0,0.0,0.0,0.0,0.5
685574,6842886,-25,0.0,1.0,0.0,0.0,0.0,0.0,0.0
685575,6842887,-31,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [47]:
ccbalance = bureau[['SK_ID_CURR','SK_ID_BUREAU']].merge(ccbalance, on = 'SK_ID_BUREAU', how= 'inner')
ccbalance

,SK_ID_CURR,SK_ID_BUREAU,Months_latest_open,latest_delinq_balance,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5
0,380361,5715448,-9,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
1,380361,5715449,-6,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
2,380361,5715451,-10,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
3,380361,5715452,-15,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
4,380361,5715453,-20,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
430282,138615,6745057,-41,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
430283,369065,6745058,-52,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
430284,369065,6745059,0,-1.0,0.857143,0.142857,0.0,0.0,0.0,0.0
430285,369065,6745060,-6,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0


In [48]:
# pick the latest open SK_ID_Bureau 
ccbalance = ccbalance.sort_values('Months_latest_open', ascending= False).groupby('SK_ID_CURR').first()
ccbalance

,SK_ID_BUREAU,Months_latest_open,latest_delinq_balance,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5
SK_ID_CURR,,,,,,,,,
100001,5896636,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
100002,6158909,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
100013,5922083,-45,-44.0,0.863636,0.136364,0.0,0.0,0.0,0.0
100019,5832096,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
100028,6356889,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
456231,5905524,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0
456234,5087533,-1,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0
456247,6022154,0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0


In [49]:
bureau_info = bureau_info.merge(ccbalance, on = 'SK_ID_CURR', how='outer')

In [50]:
bureau_info.fillna(0,inplace= True)

In [51]:
bureau_info

,AMT_CREDIT_SUM_DEBT_count_x,AMT_CREDIT_SUM_DEBT_sum_x,AMT_CREDIT_SUM_DEBT_mean_x,AMT_weightdebt_duration_sum,AMT_weightdebt_duration_mean,AMT_Overdue_duration_sum,AMT_Overdue_duration_mean,AMT_Maxoverdue_duration_mean,Days_early_paidoff_sum,Days_early_paidoff_mean,...,AMT_CREDIT_SUM_DEBT_first,SK_ID_BUREAU,Months_latest_open,latest_delinq_balance,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,7,596686.50,85240.928571,432.612431,61.801776,0.0,0.0,0.000000,3879.0,554.142857,...,0.0,5896636.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
100002,6,0.00,0.000000,0.000000,0.000000,0.0,0.0,3.852765,282.0,47.000000,...,0.0,6158909.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
100003,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1114.0,278.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004,2,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,88.0,44.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100008,2,240057.00,120028.500000,437.262295,218.631148,0.0,0.0,0.000000,646.0,323.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456243,5,1247454.00,249490.800000,8100.350649,1620.070130,0.0,0.0,0.000000,-299.0,-59.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456247,7,2193390.00,313341.428571,200.035568,28.576510,0.0,0.0,25.283572,13921.0,1988.714286,...,0.0,6022154.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
456249,4,0.00,0.000000,0.000000,0.000000,0.0,0.0,17.348901,112.0,28.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
bureau_info.to_csv('bureau_info.csv')